In [1]:
%matplotlib widget

In [2]:
import os, re
import numpy as np
import h5py as h5
import matplotlib.pyplot as plt
from scipy.constants import m_e, c
from rswarp.run_files.delta_f import delta_f_tools as dft

# Helper Functions

In [3]:
def bin_data(data, bins=32):
    heights, edges = np.histogram(data, bins=bins)
    centers = [(edges[i] + edges[i + 1]) / 2. for i in range(len(edges)-1)]
    return heights, centers

def get_rms_warp(diags):
    data = os.listdir(diags)
    sx, sy, sz, step = [], [], [], []
    for file in data:
        if file[0] == '.':
            continue
        path = os.path.join(diags, file)
        coord = h5.File(path, 'r')
        stepn = int(re.findall(r'\d+', file)[0])
        xf_warp = coord['data/{}/particles/Electron/position/x'.format(stepn)][()]
        yf_warp = coord['data/{}/particles/Electron/position/y'.format(stepn)][()]
        zf_warp = coord['data/{}/particles/Electron/position/z'.format(stepn)][()]
        sx.append(np.std(xf_warp))
        sy.append(np.std(yf_warp))
        sz.append(np.std(zf_warp))
        step.append(stepn)
        coord.close()
        
    return sx, sy, sz, step

# Physical Settings

In [4]:
L_mod = 3.7  # m

In [5]:
gamma0 = 42.66  # assumed exact
beta0 = np.sqrt(1. - 1. / (gamma0 * gamma0))

In [6]:
alpha_x_ini = 0.0
alpha_y_ini = 0.0
beta_x_ini = 4.5  # m
beta_y_ini = 4.5  # m
eps_n_rms_x = 5.0e-6  # m-rad, normalized rms emittance
eps_n_rms_y = 5.0e-6  # m-rad, normalized rms emittance
eps_rms_x = eps_n_rms_x / (gamma0 * beta0)
eps_rms_y = eps_n_rms_y / (gamma0 * beta0)

# Plotting

In [7]:
diagnostic_path = 'diags_10/hdf5/'
final_step =  5000

In [11]:
warpf = h5.File(os.path.join(diagnostic_path, 'data{:08d}.h5'.format(final_step)), 'r')

In [12]:
xf_warp = warpf['data/{}/particles/Electron/position/x'.format(final_step)][()]
yf_warp = warpf['data/{}/particles/Electron/position/y'.format(final_step)][()]
zf_warp = warpf['data/{}/particles/Electron/position/z'.format(final_step)][()]
vxf_warp = warpf['data/{}/particles/Electron/momentum/x'.format(final_step)][()]
vyf_warp = warpf['data/{}/particles/Electron/momentum/y'.format(final_step)][()]
vzf_warp = warpf['data/{}/particles/Electron/momentum/z'.format(final_step)][()]

In [13]:
dt =  warpf['data/{}'.format(final_step)].attrs['dt']

In [14]:
warpf.close()

### Beam Expansion

In [15]:
sx, sy, sz, step = get_rms_warp(diagnostic_path)

In [16]:
s = np.linspace(0, L_mod, 500)
beta_x = np.array([dft.drift_twiss(l, beta_x_ini, alpha_x_ini)[1] for l in s])

In [17]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
ax1.plot(s,
         np.sqrt(beta_x * eps_rms_x), label='benchmark: Sx', lw=4)
ax1.plot(np.array(step)*dt*beta0*c * gamma0, 
         sx, '.', label='Warp: Sx', ms=8)
ax1.legend()
ax1.set_ylabel('$\sigma_x$ (m)')
ax1.set_xlabel('$s_{lab}$ (m)')

# Assume symmetric 
ax2.plot(s,
         np.sqrt(beta_x * eps_rms_y), label='benchmark: Sy', lw=4)
ax2.plot(np.array(step)*dt*beta0*c * gamma0, 
         sy, '.', label='Warp: Sy', ms=8)
ax2.set_ylabel('$\sigma_y$ (m)')
ax2.set_xlabel('$s_{lab}$ (m)')
ax2.legend()
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Final distribution

In [18]:
fig, ax = plt.subplots(2, 3, figsize=(16, 8))

wh, wc = bin_data(xf_warp, bins=64)
ax[0,0].plot(wc, wh, label='Warp: Xf', lw=5)
ax[0,0].legend()
ax[0,0].set_xlabel('x (m)')

wh, wc = bin_data(yf_warp, bins=64)
ax[0,1].plot(wc, wh, label='Warp: Yf', lw=5)
ax[0,1].legend()
ax[0,1].set_xlabel('y (m)')

wh, wc = bin_data(zf_warp, bins=64)
ax[0,2].plot(wc, wh, label='Warp: Zf', lw=5)
ax[0,2].legend()
ax[0,2].set_xlabel('z (m)')

wh, wc = bin_data(vxf_warp / (m_e * c), bins=64)
ax[1,0].plot(wc, wh, label='Warp: VXf', lw=5)
ax[1,0].legend()
ax[1,0].set_xlabel(r'$\beta_x$ ( )')

wh, wc = bin_data(vyf_warp / (m_e * c), bins=64)
ax[1,1].plot(wc, wh, label='Warp: VYf', lw=5)
ax[1,1].legend()
ax[1,1].set_xlabel(r'$\beta_y$ ( )')

wh, wc = bin_data(vzf_warp / (m_e * c), bins=64)
ax[1,2].plot(wc, wh, label='Warp: Zf', lw=5)
ax[1,2].legend()
ax[1,2].set_xlabel(r'$\beta_y$ ( )')



plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Weighted Plots

In [19]:
# New file loading with option to compare multiple datasets in the 1D projection
# Add path to `paths`
step_num = 5000
paths = [os.path.join(diagnostic_path, 'data{:08d}.h5'.format(step_num)), ]

In [20]:
particle_data = []
for p in paths:
    fh5 = h5.File(p, 'r')
    x_warp = fh5['data/{}/particles/Electron/position/x'.format(step_num)][()]
    y_warp = fh5['data/{}/particles/Electron/position/y'.format(step_num)][()]
    z_warp = fh5['data/{}/particles/Electron/position/z'.format(step_num)][()]

    w_warp = fh5['data/{}/particles/Electron/weighting'.format(step_num)][()]
    new_p = np.array([x_warp, y_warp, z_warp, w_warp]).T
    particle_data.append(new_p.copy())
    
vel_particle_data = []
for p in paths:
    fh5 = h5.File(p, 'r')
    vx_warp = fh5['data/{}/particles/Electron/momentum/x'.format(step_num)][()]/ (m_e * c)
    vy_warp = fh5['data/{}/particles/Electron/momentum/y'.format(step_num)][()]/ (m_e * c)
    vz_warp = fh5['data/{}/particles/Electron/momentum/z'.format(step_num)][()]/ (m_e * c)

    w_warp = fh5['data/{}/particles/Electron/weighting'.format(step_num)][()]
    new_p = np.array([vx_warp, vy_warp, vz_warp, w_warp]).T
    vel_particle_data.append(new_p.copy())

In [21]:
for dat in particle_data:
    print('Total macroparticles: {}'.format(dat[:,-1].size))
    print('Total weight: {}'.format(np.sum(dat[:, -1])))
    print('Weight per particle: {}\n'.format(np.sum(dat[:, -1]) / dat[:,-1].size))

Total macroparticles: 2621440
Total weight: -0.049132405496546475
Weight per particle: -1.8742525290125456e-08



#### 1D Projections

In [22]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(14, 9))
fig.suptitle('Projected Density Distributions Weighted by $\delta - f$  Weights\nNormalized by Individual Maximum')


bins = 64#int(dat.shape[0] / base_particles_per_bin)
for i, dat in enumerate(particle_data):
    ptcl_per_bin = dat.shape[0] / bins
    heights, edges = np.histogram(dat[:, 0], weights=dat[:, -1], bins=bins)
    centers = np.array([(edges[i] + edges[i + 1]) / 2. for i in range(len(edges)-1)])
    ax1.plot(centers * 1e3, heights / np.max(heights),label='{:1.2e}    {:3}    {:5.0f}'.format(dat.shape[0], bins, ptcl_per_bin), lw=7 - 2 * i)
    ax1.set_xlabel('x (mm)')
    ax1.legend(title='    Macroparticles | Bins | MP/Bin', fontsize=8)


    heights, edges = np.histogram(dat[:,1], weights=dat[:,-1], bins=bins)
    centers = np.array([(edges[i] + edges[i + 1]) / 2. for i in range(len(edges)-1)])
    ax2.plot(centers * 1e3, heights / np.max(heights),label='{:1.2e}    {:3}    {:5.0f}'.format(dat.shape[0], bins, ptcl_per_bin), lw=7 - 2 * i)
    ax2.set_xlabel('y (mm)')
    ax2.legend(title='    Macroparticles | Bins | MP/Bin', fontsize=8)


    heights, edges = np.histogram(dat[:,2], weights=dat[:,-1], bins=bins)
    centers = np.array([(edges[i] + edges[i + 1]) / 2. for i in range(len(edges)-1)])
    ax3.plot(centers * 1e3, heights / np.max(heights), label='{:1.2e}    {:3}    {:5.0f}'.format(dat.shape[0], bins, ptcl_per_bin), lw=7 - 2 * i)
    ax3.set_xlabel('z (mm)')
    ax3.legend(title='    Macroparticles | Bins | MP/Bin', fontsize=8)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### 2D projection

In [23]:
distr = particle_data[-1]
vdistr = vel_particle_data[-1]

In [24]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4), gridspec_kw={
                           'width_ratios': [1, 1, 1.21]})
i = 0
z, x, y = np.histogram2d(distr[:, i],
           vdistr[:, i], bins=20, weights=distr[:, -1])
ax1.imshow(z, cmap='coolwarm', interpolation='gaussian', origin='lower', aspect='auto',
           extent=(np.min(distr[:, i]),np.max(distr[:, i]), np.min(vdistr[:, i]),np.max(vdistr[:, i])))
ax1.set_xlabel('x (mm)')
ax1.set_ylabel(r'$\beta_x$ ()')

i = 1
z, x, y = np.histogram2d(distr[:, i],
           vdistr[:, i], bins=20, weights=distr[:, -1])
ax2.imshow(z, cmap='coolwarm', interpolation='gaussian', origin='lower', aspect='auto',
           extent=(np.min(distr[:, i]),np.max(distr[:, i]), np.min(vdistr[:, i]),np.max(vdistr[:, i])))
ax2.set_xlabel('y (mm)')
ax2.set_ylabel(r'$\beta_y$ ()')


i = 2
z, x, y = np.histogram2d(distr[:, i],
           vdistr[:, i], bins=20, weights=distr[:, -1])
aa3 = ax3.imshow(z, cmap='coolwarm', interpolation='gaussian', origin='lower', aspect='auto',
           extent=(np.min(distr[:, i]),np.max(distr[:, i]), np.min(vdistr[:, i]),np.max(vdistr[:, i])))
ax3.set_xlabel('z (mm)')
ax3.set_ylabel(r'$\beta_z$ ()')


clb = plt.colorbar(aa3)
clb.ax.set_title('Density\n(arb. units)')
plt.tight_layout()

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …